In [1]:
import pandas as pd, numpy as np
import geopandas as gp
from geopy.geocoders import GoogleV3
from shapely import wkt
# from shapely.geometry import Polygon, Point
from geo_df import GeoDF
import df_util
from df_util import head
input_path = lambda name: f'../input-data/{name}.csv'
work_path = lambda name: f'../working-data/{name}.csv'

district_boundaries = pd.read_csv(input_path('school_district_boundaries__esrk-9vjd'))
county_boundaries = pd.read_csv(input_path('county_boundaries__67vn-ijga'))
dist_grad_rate = pd.read_csv(work_path('dist_grad_rate'))
dist_grad_rate

,county,dist,students_with_disabilities_final_grad_base,students_with_disabilities_graduates_total,students_with_disabilities_graduation_rate,students_with_disabilities_completers_total,students_with_disabilities_completion_rate,limited_english_proficient_final_grad_base,limited_english_proficient_graduates_total,limited_english_proficient_graduation_rate,...,homeless_final_grad_base,homeless_graduates_total,homeless_graduation_rate,homeless_completers_total,homeless_completion_rate,gifted_talented_final_grad_base,gifted_talented_graduates_total,gifted_talented_graduation_rate,gifted_talented_completers_total,gifted_talented_completion_rate
0,STATE TOTAL,STATE TOTAL,5775.0,3099.0,53.7,3222.0,55.8,6171.0,3289.0,53.3,...,2394.0,1175.0,49.1,1262.0,52.7,6604.0,6048.0,91.6,6156.0,93.2
1,ADAMS,MAPLETON 1,49.0,18.0,36.7,19.0,38.8,219.0,73.0,33.3,...,41.0,12.0,29.3,16.0,39.0,44.0,27.0,61.4,27.0,61.4
2,ADAMS,ADAMS 12 FIVE STAR SCHOOLS,250.0,118.0,47.2,127.0,50.8,379.0,257.0,67.8,...,106.0,62.0,58.5,65.0,61.3,227.0,201.0,88.5,208.0,91.6
3,ADAMS,ADAMS COUNTY 14,59.0,32.0,54.2,32.0,54.2,170.0,86.0,50.6,...,99.0,52.0,52.5,57.0,57.6,30.0,27.0,90.0,27.0,90.0
4,ADAMS,BRIGHTON 27J,66.0,33.0,50.0,35.0,53.0,110.0,63.0,57.3,...,41.0,21.0,51.2,21.0,51.2,63.0,51.0,81.0,52.0,82.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,DENVER,CHARTER SCHOOL INSTITUTE,111.0,16.0,14.4,17.0,15.3,141.0,31.0,22.0,...,121.0,11.0,9.1,11.0,9.1,47.0,23.0,48.9,27.0,57.4
180,CHAFFEE,MOUNTAIN BOCES,19.0,14.0,73.7,16.0,84.2,17.0,10.0,58.8,...,14.0,9.0,64.3,10.0,71.4,2.0,1.0,50.0,1.0,50.0
181,WELD,CENTENNIAL BOCES,6.0,1.0,16.7,1.0,16.7,4.0,1.0,25.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182,LA PLATA,SAN JUAN BOCES,1.0,0.0,0.0,0.0,0.0,1.0,1.0,100.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0


In [12]:
school_county = (dist_grad_rate
    [['county', 'dist']]
    [dist_grad_rate.county != "STATE TOTAL"]
)
head(school_county)

2 cols x 183 rows


,county,dist
1,ADAMS,MAPLETON 1
2,ADAMS,ADAMS 12 FIVE STAR SCHOOLS
3,ADAMS,ADAMS COUNTY 14


In [13]:
county = (county_boundaries
    [['the_geom', 'COUNTY']]
    .rename(columns={'the_geom':'geo', 'COUNTY': 'county'})
)
county.to_csv(work_path('geo_county'), index=False)
county = county.rename(columns={'geo':'geo_county'})
head(county)

2 cols x 64 rows


,geo_county,county
0,MULTIPOLYGON (((-103.70574149517748 ...,ADAMS
1,MULTIPOLYGON (((-105.59917426201822 ...,ALAMOSA
2,MULTIPOLYGON (((-103.70653410023402 ...,ARAPAHOE


In [14]:
district = (district_boundaries
    [['the_geom', 'abbrev_nam', 'mail_addre', 'mail_city', 'mail_state', 'mail_zip']]
    .set_columns(['geo_dist', 'dist', 'street', 'city', 'state', 'zip'])
)
head(district)

6 cols x 178 rows


,geo_dist,dist,street,city,state,zip
0,MULTIPOLYGON (((-106.59904239399998 ...,Buena Vista R-31 School District,PO Box 2027,Buena Vista,CO,81211
1,MULTIPOLYGON (((-102.43672348799998 ...,Burlington RE-6J School District,PO Box 369,Burlington,CO,80807
2,MULTIPOLYGON (((-104.28505092499995 ...,Byers 32J School District,444 East Front Street,Byers,CO,80103


In [15]:
df = district.copy()

df.dist = (df.dist
    .str.lower()
    .str.replace(" school district", "")
)

df = df[ ~ df.dist.isin([ 'consolidated c-1', 'florence re-2', 'gilcrest re-1'])]

district_renaming_map = {
    'bayfield 10 jt.-r': 'bayfield 10 jt-r',
    'buffalo re-4': 'buffalo re-4j',
    'creede consolidated 1': 'creede school district',
    'dolores county re no. 2': 'dolores county re no.2',
    'gunnison re1j': 'gunnison watershed re1j',
    'keenesburg re-3j': 'keenesburg re-3(j)',
    'mcclave re-2': 'mc clave re-2',
    'pueblo city schools': 'pueblo city 60',
    'pueblo county rural 70': 'pueblo county 70',
    'st. vrain valley re 1j': 'st vrain valley re 1j',
    'thompson r-2j': 'thompson r2-j',
    'weld county re-8': 'weld county s/d re-8',
    'debeque 49jt': 'de beque 49jt',
    'lewis palmer 38': 'lewis-palmer 38',
    'north park r-1': 'north park r-1 ',
}

for old, new in district_renaming_map.items():
    df.dist = df.dist.str.replace(old, new, regex=False)

df.dist = df.dist.str.upper()

district = df
head(district)

6 cols x 175 rows


,geo_dist,dist,street,city,state,zip
0,MULTIPOLYGON (((-106.59904239399998 ...,BUENA VISTA R-31,PO Box 2027,Buena Vista,CO,81211
1,MULTIPOLYGON (((-102.43672348799998 ...,BURLINGTON RE-6J,PO Box 369,Burlington,CO,80807
2,MULTIPOLYGON (((-104.28505092499995 ...,BYERS 32J,444 East Front Street,Byers,CO,80103


In [16]:
pd.options.display.max_colwidth = 40
df = (school_county
    .merge(district, on='dist', how='left')
    .merge(county, on='county', how='left')
)
head(df)

8 cols x 183 rows


,county,dist,geo_dist,street,city,state,zip,geo_county
0,ADAMS,MAPLETON 1,MULTIPOLYGON (((-105.01581612299998 ...,5910 East 80th Avenue,Denver,CO,80229,MULTIPOLYGON (((-103.70574149517748 ...
1,ADAMS,ADAMS 12 FIVE STAR SCHOOLS,MULTIPOLYGON (((-105.05310614499996 ...,1500 E 128th Avenue,Thornton,CO,80241,MULTIPOLYGON (((-103.70574149517748 ...
2,ADAMS,ADAMS COUNTY 14,MULTIPOLYGON (((-104.96883410999999 ...,5291 East 60th Avenue,Commerce City,CO,80022,MULTIPOLYGON (((-103.70574149517748 ...


In [17]:
raise Exception("This is here to keep you from accidentally running the below cell when using 'Run All'")

In [18]:
df['address'] = df.street + ', ' + df.city + ', ' + df.state + ', ' + df.zip

df['geo_dist_point'] = np.NaN
google_api_kwargs = dict(provider='google', api_key='NVmnFlQnc_MiOFcCoz7dwIFmVgq5f1zncXCdwJp')

df.loc[ ~ df.address.isna(), 'geo_dist_point'] = gp.tools.geocode(df.address, **google_api_kwargs)['geometry']

df['geo_county_point'] = gp.tools.geocode(df.county.str.capitalize() + " County, Colorado", **google_api_kwargs)['geometry']

df = df[['county', 'dist', 'geo_county', 'geo_dist', 'geo_county_point', 'geo_dist_point', 'address', 'street', 'city', 'state', 'zip']]
display(df)

11 cols x 183 rows


,county,dist,geo_county,geo_dist,geo_county_point,geo_dist_point,address,street,city,state,zip
0,ADAMS,MAPLETON 1,MULTIPOLYGON (((-103.70574149517748 ...,MULTIPOLYGON (((-105.01581612299998 ...,POINT (-104.19309 39.83983),POINT (-104.9187196 39.8415103),"5910 East 80th Avenue, Denver, CO, 8...",5910 East 80th Avenue,Denver,CO,80229
1,ADAMS,ADAMS 12 FIVE STAR SCHOOLS,MULTIPOLYGON (((-103.70574149517748 ...,MULTIPOLYGON (((-105.05310614499996 ...,POINT (-104.19309 39.83983),POINT (-104.9668135 39.9262994),"1500 E 128th Avenue, Thornton, CO, 8...",1500 E 128th Avenue,Thornton,CO,80241
2,ADAMS,ADAMS COUNTY 14,MULTIPOLYGON (((-103.70574149517748 ...,MULTIPOLYGON (((-104.96883410999999 ...,POINT (-104.19309 39.83983),POINT (-104.9268419 39.8059605),"5291 East 60th Avenue, Commerce City...",5291 East 60th Avenue,Commerce City,CO,80022


In [19]:
df.to_csv(work_path('geo_county_school'), index=False)